In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import pathlib
import cv2
import seaborn as sns

In [ ]:
# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU is available')
else:
    print('No GPU detected')

num_gpus = len(physical_devices)

if num_gpus > 0:
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {tf.config.experimental.get_device_details(physical_devices[0])}")
else:
    print("No GPUs available")

device = tf.device('gpu:0' if len(physical_devices) > 0 else 'cpu:0')

In [ ]:
import os
os.getcwd()

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random
from skimage import exposure

# Directory containing MRI images
data_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer raw MRI data\\Brain_Cancer'
save_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer_processed_2'  # Directory to save preprocessed images

# Create directory to save preprocessed images
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Function to apply intensity normalization (Z-score normalization)
def normalize_image(image):
    mean = np.mean(image)
    std = np.std(image)
    normalized_image = (image - mean) / std
    return normalized_image

# Function for data augmentation
def augment_image(image):
    aug_images = []

    # Flipping horizontally
    flipped_image = cv2.flip(image, 1)
    aug_images.append(flipped_image)

    # Adding noise
    noise = np.random.normal(0, 0.0023**0.5, image.shape)
    noisy_image = image + noise
    aug_images.append(np.clip(noisy_image, 0, 1))  # Ensure pixel values are in [0, 1]

    # Adjusting contrast and brightness using histogram equalization
    contrast_image = exposure.equalize_hist(image)  # Histogram equalization to enhance contrast
    aug_images.append(contrast_image)

    # Rotations
    rows, cols = image.shape[:2]
    for angle in [-13, -9, +9, +13]:
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated_image = cv2.warpAffine(image, M, (cols, rows))
        aug_images.append(rotated_image)

    # Apply random affine transformation
    for _ in range(2):  # Generate two affine transformations
        pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
        pts2 = pts1 + np.random.uniform(-10, 10, size=pts1.shape).astype(np.float32)
        M_affine = cv2.getAffineTransform(pts1, pts2)
        affine_image = cv2.warpAffine(image, M_affine, (cols, rows))
        aug_images.append(affine_image)

    return aug_images

# Loop through all subfolders and images
for subfolder in os.listdir(data_dir):
    subfolder_path = os.path.join(data_dir, subfolder)
    
    # Ensure the path is a directory
    if os.path.isdir(subfolder_path):
        # Create corresponding subfolder in the save directory
        save_subfolder = os.path.join(save_dir, subfolder)
        if not os.path.exists(save_subfolder):
            os.makedirs(save_subfolder)
        
        # Loop through all images in the current subfolder
        for img_name in tqdm(os.listdir(subfolder_path)):
            # Read the image
            img_path = os.path.join(subfolder_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if image is None:
                continue

            # Step 1: Resize to 256x256
            resized_image = cv2.resize(image, (256, 256))

            # Step 2: Normalize (Z-score normalization)
            normalized_image = normalize_image(resized_image)

            # Step 3: Apply augmentation
            augmented_images = augment_image(normalized_image)

            # Save the processed images
            base_name, ext = os.path.splitext(img_name)
            cv2.imwrite(os.path.join(save_subfolder, f"{base_name}_resized{ext}"), (normalized_image * 255).astype(np.uint8))

            # Save augmented images
            for i, aug_image in enumerate(augmented_images):
                cv2.imwrite(os.path.join(save_subfolder, f"{base_name}_aug_{i}{ext}"), (aug_image * 255).astype(np.uint8))

print(f"Preprocessing complete. Processed images saved to {save_dir}.")


In [ ]:
data_dir = 'D:\Brain Cancer\Dataset\Brain_Cancer_processed3'
os.listdir(data_dir)

In [ ]:
tf.random.set_seed(42)

train_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'training', seed = 1, shuffle = True, batch_size = 128, image_size=(256,256))

test_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'validation', seed = 1, shuffle = True, batch_size = 128, image_size=(256,256))

In [ ]:
filenames = pathlib.Path(data_dir)
for label in train_data.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
train_data.cardinality().numpy(),  test_data.cardinality().numpy()

In [ ]:
train_set = train_data.take(152)
val_set = train_data.skip(152)

In [ ]:
train_set.cardinality().numpy(), val_set.cardinality().numpy()

In [ ]:
# print random images from the train set
plt.figure(figsize = (15, 15))
for images, labels in train_set.take(1):
    for i in range(15):
        index = random.randint(0, len(images))
        ax = plt.subplot(3, 5, i + 1)
        plt.imshow(images[index].numpy().astype("uint8"))
        plt.title(train_data.class_names[labels[index]], color= 'blue', fontsize= 12)
        plt.axis(True)
plt.show()

In [ ]:
for images_batch, labels_batch in train_set:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Flatten
from tensorflow.keras import Model, Input

In [ ]:
tf.random.set_seed(42)

# Define the input layer
input_layer = Input(shape=(256, 256, 3))

# Convolutional layers with max pooling
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Flatten the output
x = Flatten()(x)

# Fully connected layers with dropout
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)

# Output layer
output_layer = Dense(3, activation='softmax')(x)

In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Print the model summary
model.summary()

In [ ]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(learning_rate=1e-4), metrics = 'accuracy')

In [ ]:
history_1 = model.fit(train_set, epochs=30, validation_data=val_set)

In [ ]:
model.summary()

In [ ]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2) 
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history_1.history))

In [ ]:
X_test, y_test = None, None
for images, labels in test_data:
    if X_test == None or y_test == None:
        X_test = images
        y_test = labels
    else:
        X_test = tf.concat([X_test, images], axis = 0)
        y_test = tf.concat([y_test, labels], axis = 0)
        
X_test.shape, y_test.shape

In [ ]:
from sklearn import metrics

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
test_score = model.evaluate(test_data, verbose= 1)

print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import classification_report
target_names = ['brain_glioma', 'brain_menin', 'brain_tumor']
print(classification_report(y_test , y_pred, target_names=target_names))

In [ ]:
plt.figure(figsize = (6,6), dpi = 100)
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot = True, fmt='d', cmap = 'Greens')
plt.xlabel('Predictions')
plt.ylabel('True Labels')
plt.title('Conusion Matrix')
plt.show()

In [ ]:
# plot random images from a given dataset, and compare predictions with ground truth
def plot_random_predictions(dataset, model):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        plt.figure(figsize = (10, 10), dpi = 120)
        y_pred_proba = model.predict(images)

    for i in range(9):
        index = random.randint(0, len(images))
        ax = plt.subplot(3,3, i + 1)

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]
      
        c = 'g' if y_pred == y_true else 'r'
      
        plt.imshow(img)
        plt.title(f'Predicted : {y_pred}\nTrue label : {y_true}', c = c)
        plt.axis(False)

In [ ]:
plot_random_predictions(test_data, model)